In [13]:
!pip install ultralytics -q
import cv2
import torch
import time
from pathlib import Path
from google.colab.patches import cv2_imshow
from ultralytics import YOLO

In [14]:
model_performance = {}

In [15]:
video_path = '/content/v.mp4'


In [16]:
for model_name in ["yolov8n", "yolo11n", "yolo12n"]:
    try:
        model = YOLO(f"{model_name}.pt")
        cap = cv2.VideoCapture(video_path)
        fps = cap.get(cv2.CAP_PROP_FPS)
        total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

        if not cap.isOpened():
            print(f"Error: Could not open video file {video_path}")
            continue

        start_time = time.time()
        frames_processed = 0

        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break
            results = model(frame, verbose=False)
            frames_processed += 1

        end_time = time.time()
        cap.release()

        inference_time = end_time - start_time
        calculated_fps = frames_processed / inference_time if inference_time > 0 else 0

        model_performance[model_name] = {
            'inference_time_seconds': inference_time,
            'processed_frames': frames_processed,
            'calculated_fps': calculated_fps
        }
        print(f"Finished processing with {model_name}")

    except Exception as e:
        print(f"Could not process with model {model_name}: {e}")
        model_performance[model_name] = {'error': str(e)}


Finished processing with yolov8n
Finished processing with yolo11n
Finished processing with yolo12n


In [17]:
print("\n--- Model Performance Comparison ---")
for model_name, metrics in model_performance.items():
    print(f"\nModel: {model_name}")
    if 'error' in metrics:
        print(f"  Status: Error - {metrics['error']}")
    else:
        print(f"  Total Inference Time: {metrics['inference_time_seconds']:.2f} seconds")
        print(f"  Frames Processed: {metrics['processed_frames']}")
        print(f"  Calculated FPS: {metrics['calculated_fps']:.2f}")
        # Print other metrics here

print("------------------------------------")


--- Model Performance Comparison ---

Model: yolov8n
  Total Inference Time: 114.43 seconds
  Frames Processed: 598
  Calculated FPS: 5.23

Model: yolo11n
  Total Inference Time: 98.98 seconds
  Frames Processed: 598
  Calculated FPS: 6.04

Model: yolo12n
  Total Inference Time: 101.36 seconds
  Frames Processed: 598
  Calculated FPS: 5.90
------------------------------------
